In [ ]:
import numpy as np

In [ ]:
# ==============================================================================
# 构造一个完整的场景数据字典
# 用于多视角物体重建、NOCS（归一化对象坐标空间）估计、6D姿态估计等任务
# ==============================================================================

scene_data = {
    # --------------------------------------------------------------------------------
    # scene_colors: 多视角下的RGB渲染图像
    # shape: (N_views, H, W, 3)
    # - N_views: 同一场景下从不同相机视角拍摄/渲染的图像数量（例如10个视角）
    # - H: 图像高度（像素），此处为480
    # - W: 图像宽度（像素），此处为640
    # - 3: 通道数，对应RGB三通道（Red, Green, Blue）
    # 数据类型: uint8，值域[0, 255]
    # --------------------------------------------------------------------------------
    'scene_colors': np.zeros((10, 480, 640, 3), dtype=np.uint8),  # 示例：全黑图像

    # --------------------------------------------------------------------------------
    # scene_points: 每个像素对应的3D世界坐标（或相机坐标）
    # shape: (N_views, H, W, 3)
    # - 3 表示 (X, Y, Z) 3D空间坐标（单位：米）
    # 数据类型: float32
    # 说明: 若某像素无有效深度，则为 (0,0,0)
    # --------------------------------------------------------------------------------
    'scene_points': np.zeros((10, 480, 640, 3), dtype=np.float32),  # 世界坐标点

    # --------------------------------------------------------------------------------
    # scene_depths: 深度图（单通道）
    # shape: (N_views, H, W)
    # - 每个像素存储从相机到表面的距离（单位：米）
    # 数据类型: float32
    # 说明: 渲染生成
    # --------------------------------------------------------------------------------
    'scene_depths': np.zeros((10, 480, 640), dtype=np.float32),  # 深度图

    # --------------------------------------------------------------------------------
    # extrinsics: 相机外参（从世界坐标系到相机坐标系的变换）
    # shape: (N_views, 4, 4)
    # - 每个是 4x4 的齐次变换矩阵
    #   [ R | t ]   R: 3x3 旋转矩阵（世界→相机）
    #   [----]     t: 3x1 平移向量（世界原点在相机坐标中的位置，单位为m）
    #   [ 0 | 1 ]
    # 数据类型: float32
    # --------------------------------------------------------------------------------
    'extrinsics': np.zeros((10, 4, 4), dtype=np.float32),  # 外参矩阵

    # --------------------------------------------------------------------------------
    # intrinsics: 相机内参矩阵
    # shape: (N_views, 3, 3)
    # 格式:
    # [ fx   0  cx ]
    # [  0  fy  cy ]
    # [  0   0   1 ]
    # - fx, fy: 焦距（像素单位）
    # - cx, cy: 主点（图像中心偏移）
    # 注意: 多数情况下所有视角共享相同内参，但仍为 (10,3,3) 以支持不同相机
    # 数据类型: float32
    # --------------------------------------------------------------------------------
    'intrinsics': np.zeros((10, 3, 3), dtype=np.float32),  # 内参矩阵

    # --------------------------------------------------------------------------------
    # scene_scales: 预定义的物体类别尺度
    # shape: (N_models, 3)
    # - N_models: 物体类别数量（例如4类）
    # - 3: 对应每个物品从nocs大小放缩为真实大小要乘以的倍数，三个倍数应当为一致
    # 示例: [[1.86382556 1.86382556 1.86382556]
    #  [0.33668262 0.33668262 0.33668262]
    #  [0.2889713  0.2889713  0.2889713 ]
    #  [0.55243242 0.55243242 0.55243242]]
    # 数据类型: float32
    # --------------------------------------------------------------------------------
    'scene_scales': np.zeros((4, 3), dtype=np.float32),  # (类别数, 3) 的尺寸模板

    # --------------------------------------------------------------------------------
    # scene_poses: 每帧图像中物体的6D姿态（SE(3) 变换矩阵， t的单位为m）
    # 类型: dict，键为图像ID字符串（如 '0000', '0001'...）
    # 值: numpy array of shape (N_objects, 4, 4)
    # - N_objects: 当前图像中检测到的实例数量, 排序按照instance_id从小到大
    # - 每个 4x4 矩阵表示从物体局部坐标系到世界坐标系的变换
    # 说明: 物体此矩阵描述其在世界中的位姿
    # 数据类型: float32
    # 示例：'0001':
    # shape: (2, 4, 4)
    # [[[-0.06867076  0.85326827 -0.5169309  -0.46486375]
    #   [-0.2547874  -0.51597095 -0.817837   -0.40760714]
    #   [-0.96455574  0.07554598  0.2528341   1.1996272 ]
    #   [ 0.          0.          0.          1.        ]]
    #  [[-0.85822576 -0.4919216   0.146498    0.00566059]
    #   [ 0.4380641  -0.55326086  0.7085212   0.0038019 ]
    #   [-0.26748526  0.67224663  0.69031596  1.0661576 ]
    #   [ 0.          0.          0.          1.        ]]]
    # --------------------------------------------------------------------------------
    'scene_poses': {
        '0000': np.zeros((1, 4, 4), dtype=np.float32),  # 图像0000中有1个物体
        '0001': np.zeros((2, 4, 4), dtype=np.float32),  # 图像0001中有2个物体
        '0002': np.zeros((2, 4, 4), dtype=np.float32),
        '0003': np.zeros((2, 4, 4), dtype=np.float32),
        '0004': np.zeros((2, 4, 4), dtype=np.float32),
        '0005': np.zeros((2, 4, 4), dtype=np.float32),
        '0006': np.zeros((2, 4, 4), dtype=np.float32),
        '0007': np.zeros((2, 4, 4), dtype=np.float32),
        '0008': np.zeros((2, 4, 4), dtype=np.float32),
        '0009': np.zeros((2, 4, 4), dtype=np.float32),
    },

    # --------------------------------------------------------------------------------
    # masks: 实例分割掩码
    # shape: (N_views, H, W)
    # - 每个像素值为整数 instance_id（255表示背景）
    # - 例如：1 表示第一个物体实例，4 表示第四个 (从1开始编号)
    # 数据类型: uint8
    # 说明: 用于区分不同物体实例
    # --------------------------------------------------------------------------------
    'masks': np.zeros((10, 480, 640), dtype=np.uint8),

    # --------------------------------------------------------------------------------
    # nocs_maps: NOCS（Normalized Object Coordinate Space）图（将原先m的坐标乘以255并变为uint8的格式，背景为255）
    # shape: (N_views, H, W, 3)
    # ✅ NOCS 的三个定义条件：
    # 1. 中心在 (0.5, 0.5, 0.5)
    # → 点云的质心（或 bounding box 中心）应接近 (0.5, 0.5, 0.5)
    # 2. tight bounding box 的对角线长度为 1
    # → 包围所有点的最小轴对齐包围盒（AABB）的对角线长度 ≈ 1
    # 3. 具有标准朝向（Standard Orientation）
    # → 通常指：物体的主方向与坐标轴对齐
    # 数据类型: uint8
    # 说明: 仅在有物体的区域有效，背景为 (255,255,255)
    # --------------------------------------------------------------------------------
    'nocs_maps': np.zeros((10, 480, 640, 3), dtype=np.uint8),

    # --------------------------------------------------------------------------------
    # scene_instance_ids: 每张图像中出现的物体实例ID列表
    # 类型: dict，键为图像ID
    # 值: 一维 numpy array，shape: (N_objects,)
    # - 存储当前图像中出现的 instance_id（与 masks 中的值对应）
    # 说明: 用于快速查询每帧有哪些物体
    # 示例: 图像 '0000' 只有 instance_id=4 的物体
    # 数据类型: uint8
    # --------------------------------------------------------------------------------
    'scene_instance_ids': {
        '0000': np.array([4], dtype=np.uint8),
        '0001': np.array([1, 4], dtype=np.uint8),
        '0002': np.array([1, 4], dtype=np.uint8),
        '0003': np.array([1, 4], dtype=np.uint8),
        '0004': np.array([1, 4], dtype=np.uint8),
        '0005': np.array([1, 4], dtype=np.uint8),
        '0006': np.array([1, 4], dtype=np.uint8),
        '0007': np.array([1, 4], dtype=np.uint8),
        '0008': np.array([1, 4], dtype=np.uint8),
        '0009': np.array([1, 4], dtype=np.uint8),
    },

    # --------------------------------------------------------------------------------
    # model_list: 所有预定义物体模型的名称列表
    # shape: (N_models,)
    # - 字符串数组，每个元素是物体类别的唯一标识符
    # - 例如来自 NOCS 数据集的模型名,顺序与前面的masks及scene_instance_ids的序号一致，从1开始
    # 数据类型: object (numpy array of strings)
    # --------------------------------------------------------------------------------
    'model_list': np.array(['mushroom_011', 'tamarind_018', 'conch_114', 'nipple_013'], dtype='object'),

    # --------------------------------------------------------------------------------
    # bbox: 每张图像中检测到的3D边界框信息
    # 类型: dict，键为图像ID（如 '0000'）
    # 值: dict，键为物体ID（如 0, 1），值为包含以下字段的 dict:
    #   - 'x', 'y', 'z': 物体中心在世界坐标系中的位置（单位：米）
    #   - 'w', 'h', 'l': 宽、高、长（单位：米）
    #   - 'R': 3x3 旋转矩阵（描述物体朝向）
    # 数据类型: float32 for 数值，np.ndarray for R
    # --------------------------------------------------------------------------------
    'bbox': {
        '0000': {
            0: {
                'x': 10.5,
                'y': 2.0,
                'z': 1.7,
                'w': 2.5,
                'h': 1.8,
                'l': 4.0,
                'R': np.array([[ 0.98,  0.00, -0.20],
                               [ 0.00,  1.00,  0.00],
                               [ 0.20,  0.00,  0.98]], dtype=np.float32)
            },
            1: {
                'x': 8.0,
                'y': 1.8,
                'z': 0.5,
                'w': 0.8,
                'h': 1.7,
                'l': 0.6,
                'R': np.eye(3, dtype=np.float32)  # 单位阵，无旋转
            },
            2: {
                'x': 15.2,
                'y': 2.1,
                'z': 0.8,
                'w': 0.8,
                'h': 1.9,
                'l': 1.8,
                'R': np.array([[ 0.71,  0.00,  0.71],
                               [ 0.00,  1.00,  0.00],
                               [-0.71,  0.00,  0.71]], dtype=np.float32)
            }
        },
        '0001': {
            0: {
                'x': 20.1,
                'y': 0.4,
                'z': 1.0,
                'w': 1.0,
                'h': 1.5,
                'l': 1.2,
                'R': np.array([[ 0.00,  0.00,  1.00],
                               [ 0.00,  1.00,  0.00],
                               [-1.00,  0.00,  0.00]], dtype=np.float32)
            },
            1: {
                'x': 12.3,
                'y': 0.6,
                'z': 5.5,
                'w': 3.0,
                'h': 2.0,
                'l': 6.0,
                'R': np.eye(3, dtype=np.float32)
            }
        }
        # 可继续扩展 '0002' 到 '0009' ...
    }
}